In [1]:
# Preamble
from astropy.table import Table, column
import numpy as np
import os

%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
# better-looking plots
plt.rcParams['font.family'] = 'serif'
plt.rcParams['figure.figsize'] = (10.0, 8)
plt.rcParams['font.size'] = 18
mpl.ticker.AutoLocator.default_params['nbins'] = 5
mpl.ticker.AutoLocator.default_params['prune'] = 'both'

mpl.rcParams['ps.useafm'] = True
mpl.rcParams['pdf.use14corefonts'] = True
mpl.rcParams['text.usetex'] = True

import sys
sys.path.append('../astro_codes/')

#### First task is to load all of the data we require

In [2]:
gz1_data = Table.read('../fits/GZ1/gz1_matched.fits')
p_cw = gz1_data['p_cw']/(gz1_data['p_ccw']+gz1_data['p_cw'])

In [3]:
sparcfire_r = Table.read('../sparcfire_directories/r/r.galaxy_matched.fits') # sparcfire data
ml_table_matched = Table.read('fits/ml_table_matched.fits')

#sparcfire_g = Table.read('../sparcfire_directories/g/g.galaxy_matched.fits')
#g_ok = Table.read('g_vs_r/g_ok.fits')

In [4]:
in_gz1 = (gz1_data['N_spiral'] >= 5) #& (gz1_data['strong_agreement'])
good_agreement = np.any([p_cw < 0.2,p_cw > 0.8],axis=0).T

sample_ok = np.all([ml_table_matched['vmax_ok'],
                    sparcfire_r['fit_state'] == 'OK'],axis=0)
print('{} ML-limited spirals in the 1/Vmax sample!'.format(sample_ok.sum()))

gz1_spirals = np.all([in_gz1,sample_ok],axis=0)
gz1_spirals_best = np.all([gz1_spirals,good_agreement],axis=0)
print('+ {} in GZ1'.format(gz1_spirals.sum()))
print('+ {} w. agreement > 0.8'.format(gz1_spirals_best.sum()))

4801 ML-limited spirals in the 1/Vmax sample!
+ 4779 in GZ1
+ 3967 w. agreement > 0.8


In [5]:
chi_gz1 = (gz1_data['chirality'] == 'Swise').astype(int)

chirality_columns = ['chirality_maj',
                     'chirality_alenWtd',
                     'chirality_wtdPangSum',
                     'chirality_longestArc']

for column in chirality_columns:
    chi_sparcfire = (sparcfire_r[column] == 'Zwise').astype(int)
    chi_agree = chi_gz1 == chi_sparcfire
    gz1_agree = chi_agree[gz1_spirals]
    gz1_agree_best = chi_agree[gz1_spirals_best]
    print('--------')
    print(column+':')
    print('{}/{} galaxies agree ({}%)'.format(gz1_agree.sum(),gz1_agree.size,
                                              np.round(gz1_agree.sum()/gz1_agree.size*100,decimals=1)))
    print('{}/{} "best" galaxies agree ({}%)'.format(gz1_agree_best.sum(),gz1_agree_best.size,
                                                     np.round(gz1_agree_best.sum()/gz1_agree_best.size*100,decimals=1)))

--------
chirality_maj:
3703/4779 galaxies agree (77.5%)
3285/3967 "best" galaxies agree (82.8%)
--------
chirality_alenWtd:
4112/4779 galaxies agree (86.0%)
3669/3967 "best" galaxies agree (92.5%)
--------
chirality_wtdPangSum:
4102/4779 galaxies agree (85.8%)
3676/3967 "best" galaxies agree (92.7%)
--------
chirality_longestArc:
3959/4779 galaxies agree (82.8%)
3534/3967 "best" galaxies agree (89.1%)


In [6]:
4779/4801

0.9954176213288898